In [1]:
!pip install smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 4.3 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from PIL import Image
import requests
from io import BytesIO

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/e/e8/The_Joker_at_Wax_Museum_Plus.jpg",
    "https://upload.wikimedia.org/wikipedia/en/9/98/Joker_%28DC_Comics_character%29.jpg"
]

images = []
for url in image_urls:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    response = requests.get(url,headers=headers)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    images.append(image)

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = ""

In [5]:
!pip install smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
import base64
import os

os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""

LANGFUSE_PUBLIC_KEY = os.environ.get("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.environ.get("LANGFUSE_SECRET_KEY")

if not LANGFUSE_PUBLIC_KEY or not LANGFUSE_SECRET_KEY:
    raise ValueError("Langfuse public or secret keys are missing!")

LANGFUSE_AUTH = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

#os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

print("Langfuse environment variables set successfully!")

Langfuse environment variables set successfully!


In [7]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [8]:
from smolagents import CodeAgent, OpenAIServerModel

model = OpenAIServerModel(model_id="gpt-4o")

# Instantiate the agent
agent = CodeAgent(
    tools=[],
    model=model,
    max_steps=20,
    verbosity_level=2
)

response = agent.run(
    """
    Describe the costume and makeup that the comic character in these photos is wearing and return the description.
    Tell me if the guest is The Joker or Wonder Woman.
    """,
    images=images
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Describe the costume and makeup that the comic character in these photos is wearing and return the description. │
│     Tell me if the guest is The Joker or Wonder Woman.                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
I can't identify people or characters in images, but I can describe the costume and makeup.                        
                                                                                                                   
The character is wearing:                                                                                          
                                                                                                                   
1. **First Image Description:**                                                                                    
   - A bright purple suit.                                                                                         
   - A mustard-yellow shirt.                                                                                       
   - A large, satiny, purple tie or cravat.                                                                        
   - White face makeup with exaggerated, wide red lips.                                                            
   - Dark eye makeup with blue tones.                                                                              
                                                                                                                   
2. **Second Image Description:**                                                                                   
   - A dark suit with a green bow tie.                                                                             
   - White face makeup with a sinister red smile.                                                                  
   - Bright green hair.                                                                                            
   - Holding a playing card with a joker on it.                                                                    
                                                                                                                   
Based on the description, these characteristics are associated with The Joker.                                     

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            I can't identify people or characters in images, but I can describe the costume and makeup.

The character is wearing:

1. **First Image Description:**
   - A bright purple suit.
   - A mustard-yellow shirt.
   - A large, satiny, purple tie or cravat.
   - White face makeup with exaggerated, wide red lips.
   - Dark eye makeup with blue tones.

2. **Second Image Description:**
   - A dark suit with a green bow tie.
   - White face makeup with a sinister red smile.
   - Bright green hair.
   - Holding a playing card with a joker on it.

Based on the description, these characteristics are associated with The Joker.</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 1: Duration 8.16 seconds| Input tokens: 2,936 | Output tokens: 141]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I'll describe the costume and makeup, ensuring the format is correct.                                     
                                                                                                                   
<code>                                                                                                             
# Description of the character in the images                                                                       
                                                                                                                   
# First Image                                                                                                      
first_image_description = """                                                                                      
1. **First Image Description:**                                                                                    
   - A bright purple suit.                                                                                         
   - A mustard-yellow shirt.                                                                                       
   - A large, satiny, purple tie or cravat.                                                                        
   - White face makeup with exaggerated, wide red lips.                                                            
   - Dark eye makeup with blue tones.                                                                              
"""                                                                                                                
                                                                                                                   
# Second Image                                                                                                     
second_image_description = """                                                                                     
2. **Second Image Description:**                                                                                   
   - A dark suit with a green bow tie.                                                                             
   - White face makeup with a sinister red smile.                                                                  
   - Bright green hair.                                                                                            
   - Holding a playing card with a joker on it.                                                                    
"""                                                                                                                
                                                                                                                   
# The character is associated with The Joker.                                                                      
character_identity = "The Joker"                                                                                   
                                                                                                                   
final_answer((first_image_description, second_image_description, character_identity))                              
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Description of the character in the images                                                                     
                                                                                                                   
  # First Image                                                                                                    
  first_image_description = """                                                                                    
  1. **First Image Description:**                                                                                  
     - A bright purple suit.                                                                                       
     - A mustard-yellow shirt.                                                                                     
     - A large, satiny, purple tie or cravat.                                                                      
     - White face makeup with exaggerated, wide red lips.                                                          
     - Dark eye makeup with blue tones.                                                                            
  """                                                                                                              
                                                                                                                   
  # Second Image                                                                                                   
  second_image_description = """                                                                                   
  2. **Second Image Description:**                                                                                 
     - A dark suit with a green bow tie.                                                                           
     - White face makeup with a sinister red smile.                                                                
     - Bright green hair.                                                                                          
     - Holding a playing card with a joker on it.                                                                  
  """                                                                                                              
                                                                                                                   
  # The character is associated with The Joker.                                                                    
  character_identity = "The Joker"                                                                                 
                                                                                                                   
  final_answer((first_image_description, second_image_description, character_identity))                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: ('\n1. **First Image Description:**\n   - A bright purple suit.\n   - A mustard-yellow shirt.\n   - A
large, satiny, purple tie or cravat.\n   - White face makeup with exaggerated, wide red lips.\n   - Dark eye makeup
with blue tones.\n', '\n2. **Second Image Description:**\n   - A dark suit with a green bow tie.\n   - White face 
makeup with a sinister red smile.\n   - Bright green hair.\n   - Holding a playing card with a joker on it.\n', 
'The Joker')

[Step 2: Duration 8.36 seconds| Input tokens: 7,293 | Output tokens: 326]

In [ ]:
response